# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

# Preproccess

## read data

In [2]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [3]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [4]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


In [5]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [6]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [7]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,race,rr,saps2,sbp,scr,sofa,use_NaHCO3,uti,wbc,weight
0,19054290,20046699,30643955,False,NaN,65,NaN,NaN,NaN,NaN,...,WHITE,NaN,34,NaN,NaN,3,0.0,0,NaN,74.50
0,14866589,20066152,33060916,False,NaN,37,NaN,NaN,NaN,NaN,...,WHITE,NaN,22,NaN,NaN,2,0.0,0,NaN,NaN
0,14849360,20152551,32817197,False,NaN,59,NaN,NaN,NaN,NaN,...,UNABLE TO OBTAIN,NaN,22,NaN,NaN,4,0.0,0,NaN,NaN
0,16171124,20167052,38239449,False,NaN,37,NaN,NaN,NaN,NaN,...,HISPANIC/LATINO - DOMINICAN,NaN,15,NaN,NaN,2,0.0,0,NaN,NaN
0,16815664,20240670,37751533,False,NaN,42,NaN,NaN,NaN,NaN,...,BLACK/AFRICAN AMERICAN,NaN,27,NaN,NaN,2,NaN,0,NaN,43.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,15517352,29801212,32814591,False,NaN,20,NaN,NaN,NaN,NaN,...,WHITE,NaN,30,NaN,NaN,1,NaN,0,NaN,NaN
0,10383045,29899941,39755932,False,NaN,58,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,1,0.0,0,NaN,59.40
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,WHITE,NaN,32,NaN,NaN,3,0.0,0,NaN,47.40
0,10912213,29911812,34040470,False,NaN,47,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,2,0.0,0,NaN,89.05


In [8]:
def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients

In [9]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [10]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.dl_prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created
Dataset 1 created
Dataset 2 created
Dataset 3 created
Dataset 4 created


In [11]:
# from constants import CATEGORICAL_MEASURES
# from utils.dl_prepare_data import patientsToNumpy


# for i, (trainPatients, testPatients) in enumerate(trainTest()):
#     if i != 3: 
#         continue
    
#     trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
#     npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
#         trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
#     )

#     testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
#     npTestX, *_ = patientsToNumpy(
#         testPatients,
#         hoursPerWindow,
#         CATEGORICAL_MEASURES,
#         columns,
#         categoryEncoder,
#         numericEncoder,
#         oulier,
#     )
#     trainY = [p.akdPositive for p in trainPatients]
#     testY = [p.akdPositive for p in testPatients]

#     dataset.append((npTrainX, trainY, npTestX, testY))

#     print(f"Dataset {i} created")
#     pass

In [12]:
import pickle


(TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

41441534

# LSTM

In [13]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features)))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

2024-05-25 13:44:54.744718: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-25 13:44:54.764275: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-25 13:44:55.177927: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
# crossfold


import numpy as np


results = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    model = createModel(npTrainX.shape[1], npTrainX.shape[2])
    
    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}
    model.load_weights(weight)
    
    trainY = np.array(trainY)
    model.fit(npTrainX, trainY, epochs=20, batch_size=32, validation_split=0.2)
    
    testY = np.array(testY)
    result = model.evaluate(npTestX, testY)
    results.append(result)
    
    print(f"Dataset {i} result: {result}")
    pass

print("Mean:", np.mean(results))
print("Std:", np.std(results))

2024-05-25 13:44:55.385434: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-25 13:44:55.400490: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/tu/codepy/hust.year2023.PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer usi

ValueError: File format not supported: filepath={0: 0.8250853242320819, 1: 1.269028871391076}. Keras 3 only supports V3 `.keras` and `.weights.h5` files, or legacy V1/V2 `.h5` files.

In [22]:
print(neg, pos)
print(len(trainY))
print(weight0)
print((1 / pos) * (len(trainY)) / 2.0)
print(weight0 + weight1)

586 381
967
0.8250853242320819
1.269028871391076
2.094114195623158


In [ ]:
nan_count = np.isnan(npTrainX).sum()
nan_count / npTrainX.size

0.037094429139883686

In [ ]:
npTrainX.shape[2]

72